### Loading pickle file


In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
import pandas as pd
import pickle

In [24]:
### Load the trained model, scaler pickle, onehot

model=load_model('model.h5')

## Load encoders and scalar
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file=file)

with open('onehot_encoder_geo.pkl','rb') as file:
    one_hot_encoder_geo=pickle.load(file=file)

with open('sscaler.pkl','rb') as file:
    scaler=pickle.load(file=file)

In [25]:
## Example Input data
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000, 
    'NumOfProducts': 2,
    'HasCrCard':1,
    'IsActiveMember':1, 
    'EstimatedSalary':50000
}

In [26]:
## One hot encoding geography column
geo_encoded=one_hot_encoder_geo.transform([[input_data['Geography']]]).toarray()

##Converting into dataframe
geo_encoded_df=pd.DataFrame(geo_encoded,columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\Dell\Desktop\Python\DS\Deep learning Project\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [27]:
## converting input data into dataframe
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [28]:
## Encode categorical Variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])

In [29]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [31]:
#combine one hot encoded column with input column
input_df=pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [32]:
## Scaling input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [33]:
## Prediction
prediction=model.predict(input_scaled)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
[[0.02997888]]


In [34]:
prediction_prob=prediction[0][0]
prediction_prob

np.float32(0.02997888)

In [35]:
if prediction_prob>0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
